# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute(""" CREATE KEYSPACE IF NOT EXISTS proj
WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor' : 1};""")

#### Set Keyspace

In [7]:
session.set_keyspace('proj')

### Now we need to create tables to run the following queries. 

## Creating queries to answer the following three questions of the data

### 1. To get the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. To get only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. To get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




##### For table songs_by_user_id, columns sessionId and itemInSession as primary key because we are going to filter with these columns.

In [8]:
try:
    session.execute(""" CREATE TABLE IF NOT EXISTS songs_by_user_session (
                        sessionId INT,
                        itemInSession INT,
                        artist VARCHAR,
                        song VARCHAR,
                        length FLOAT,
                        PRIMARY KEY (sessionId, itemInSession));""")
except Exception as e:
    print(e)    

                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO songs_by_user_session (sessionId,itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]),line[0], line[9], float(line[5])))

In [10]:
result = session.execute(""" SELECT artist,length,song FROM songs_by_user_session WHERE sessionId = 338 AND itemInSession = 4;""")
df=result._current_rows
df=pd.DataFrame(df)
df

,artist,length,song
0,Faithless,495.307312,Music Matters (Mark Knight Dub)


##### For table songs_by_user_id,  userId and sessionId are primary keys as we are filtering with it. Column itemInSession is clustering key which helps us to sort the resulting table.

In [11]:
try:
    session.execute(""" CREATE TABLE IF NOT EXISTS songs_by_user_id(
                        userId INT,
                        sessionId INT,
                        itemInSession INT,
                        artist VARCHAR,
                        song VARCHAR,
                        firstName VARCHAR,
                        lastName VARCHAR,
                        PRIMARY KEY ((userId,sessionId),itemInSession));""")
except Exception as e:
    print(e)    

                    

In [12]:

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_user_id (userId, sessionId,itemInSession, artist, song, firstName,lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s,%s,%s);"
        session.execute(query, (int(line[10]),int(line[8]), int(line[3]),line[0], line[9],line[1], line[4]))

In [13]:
result = session.execute(""" SELECT artist,song, firstName,lastName FROM songs_by_user_id 
                            WHERE sessionId = 182 AND userId = 10
                            ORDER BY itemInSession;""")
df=result._current_rows
df=pd.DataFrame(df)
df

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


##### For table songs_by_song, song is primary key as we are filtering using it. Column userId is clustering column as it helps us in creating a unique key.

In [14]:
try:
    session.execute(""" CREATE TABLE IF NOT EXISTS songs_by_song(
                        song VARCHAR,
                        userId INT,
                        firstName VARCHAR,
                        lastName VARCHAR,
                        PRIMARY KEY (song,userId));""")
except Exception as e:
    print(e)    


                    

In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO songs_by_song (song, firstName,lastName,userId) "
        query = query + "VALUES (%s, %s, %s,%s);"
        session.execute(query, ( line[9],line[1], line[4], int(line[10])))

In [16]:
result= session.execute("""SELECT firstName, lastName 
                           FROM songs_by_song 
                           WHERE song = 'All Hands Against His Own'
                           ORDER BY userId""")
df=result._current_rows
df=pd.DataFrame(df)
df

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Droping the tables before closing out the sessions

In [17]:
session.execute("""DROP TABLE songs_by_user_session;""")
session.execute("""DROP TABLE songs_by_user_id;""")
session.execute("""DROP TABLE songs_by_song;""")

### Closing the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()